In [1]:
%load_ext autoreload
%autoreload 2


%matplotlib inline
%cd C:\MAD4AG

from tqdm.auto import tqdm
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

C:\MAD4AG


In [2]:
filename = r'.\dbs\twins\multiple_days_03-21.pkl'

combined_df= pd.read_pickle(filename)
#combined_df.columns

In [3]:
df_clusters=pd.read_parquet(f'./dbs/intermediate/df_selected_clusters_activities_03-21.parquet')

In [4]:
df_survey_act = pd.read_pickle(f'./dbs/intermediate/survey_act_weekday_modified.pkl')

## generate activity schedule for simulation #1
- by merging survey activity sequence table
- the survey activity sequence table brings activity sequences and activity start end times

In [5]:
day = '3'
%store -r day


no stored variable or alias day


In [6]:
df_act_schedule_0 = pd.merge(combined_df[['uid', 'employee', 'Deso', 'wt', 'wt_p', 'sub_id_'+day,'act_seq_simp_'+day]], df_survey_act[['sub_id', 'purpose','h_s','h_e']], left_on='sub_id_'+day,right_on='sub_id')

df_act_schedule_0['act_seq'] = df_act_schedule_0.groupby('uid').cumcount()



### add sequence information of other activities

In [7]:
df_act_schedule_0['other_order']= 0

df_act_schedule_0['other_order'][df_act_schedule_0.purpose=='Other'] = df_act_schedule_0[df_act_schedule_0.purpose=='Other'].groupby('uid').cumcount()

df_act_schedule_0['other_order']=df_act_schedule_0['other_order'].astype(str)

df_act_schedule_0['purpose'][df_act_schedule_0.purpose=='Other']= df_act_schedule_0['purpose'][df_act_schedule_0.purpose=='Other'] + df_act_schedule_0['other_order'][df_act_schedule_0.purpose=='Other']

df_act_schedule_0.drop(columns=['other_order'] ,inplace=True)


## re-organise df_cluster table by the other activities order and repetition

In [8]:
df_clusters_0 = df_clusters[['uid','cluster','other_clusters_'+day,'selection_'+day, 'X','Y','cluster_lat','cluster_lng']][df_clusters['selection_'+day] != 'non']

In [9]:
df_clusters_0_other = df_clusters_0[df_clusters_0['selection_'+day] =='Other']


df_other_org = df_clusters_0_other[['uid', 'other_clusters_'+day]]
df_other_org= df_other_org.drop_duplicates(subset='uid')


In [10]:
# it will transform each element of the list in the cloumn 'other_cluster_O' to a row.

df_other_org = df_other_org.explode('other_clusters_'+day)

df_other_org.rename(columns={'other_clusters_'+day:'cluster'}, inplace=True)

df_other_org['other_order'] = df_other_org.groupby('uid').cumcount()

In [11]:
df_other_org = pd.merge(df_other_org, df_clusters_0_other, on=['uid', 'cluster'],how='left')

# add sequence information of other activities

df_other_org['other_order'] = df_other_org['other_order'].astype(str)

df_other_org['selection_'+day]= df_other_org['selection_'+day]+ df_other_org['other_order']

df_other_org.drop(columns=['other_order'] ,inplace=True)

In [12]:
df_clusters_0 = df_clusters_0[df_clusters_0['selection_'+day] !='Other']

In [13]:
df_clusters_0 = pd.concat([df_clusters_0, df_other_org], ignore_index=True)

In [14]:
df_clusters_0.drop(columns=['other_clusters_'+day] ,inplace=True)

In [15]:
df_act_schedule_0 = pd.merge(df_act_schedule_0, df_clusters_0, left_on=['uid','purpose'],right_on=['uid','selection_'+day], how='left')

In [16]:
df_act_schedule_0.drop(columns=['selection_'+day] ,inplace=True)


In [17]:
df_act_schedule_0.rename(columns={'cluster_lat':'lat','cluster_lng':'lng'}, inplace=True)

In [18]:
df_act_schedule_0['purpose'][ ~df_act_schedule_0['purpose'].isin(['Home', 'Work'])] = df_act_schedule_0['purpose'][ ~df_act_schedule_0['purpose'].isin(['Home', 'Work'])].str[:5]

In [19]:
# file = r'.\dbs\twins\%s-schedule-%s.pkl'%(filename[26:31],day)
#
# df_act_schedule_0.to_pickle(file)

### activity schedules are ready here. But other activities are not ordered by distances

In [20]:
df_clusters_distances = df_clusters[['uid','cluster','other_clusters_'+day,'selection_'+day, 'X','Y','cluster_lat','cluster_lng','distance', 'work_dist']][df_clusters['selection_'+day] != 'non']

In [21]:
# from scipy.spatial import distance
# from tqdm.auto import tqdm
#
#
#
# def other_dist(data):
#
#     if len(data[data['selection_'+day] == 'Work'])>0:
#         work_dist = distance.cdist(data[['X', 'Y']][data['selection_'+day] == 'Work'].head(1), data[['X', 'Y']],  metric='euclidean')
#         data['work_dist']= work_dist[0]
#
#     return data
#
#
# tqdm.pandas()
# df_clusters_distances = df_clusters_distances.groupby('uid').progress_apply(other_dist)

In [22]:
# df_clusters_distances['work_dist']= df_clusters_distances['work_dist'].fillna(0)

In [23]:
df_clusters_distances_other = df_clusters_distances[df_clusters_distances['selection_'+day]=='Other']

In [286]:
df_clusters_distances_other = df_clusters_distances_other[['uid','other_clusters_'+day]].drop_duplicates(subset=['uid'])
df_clusters_distances_other = df_clusters_distances_other.explode('other_clusters_'+day)

In [287]:
df_clusters_distances_other = pd.merge(df_clusters_distances_other , df_clusters_distances[['uid','cluster', 'distance', 'work_dist']], left_on=['uid','other_clusters_'+day ],right_on=['uid','cluster'], how='left')

In [288]:
df_clusters_distances_other_home= df_clusters_distances_other.sort_values(by=['uid','distance'])
df_clusters_distances_other_home= df_clusters_distances_other_home.groupby('uid')['cluster'].apply(list).reset_index(name='other_cluster_dist')

In [289]:
df_clusters_distances_other_work= df_clusters_distances_other.sort_values(by=['uid','work_dist'])
df_clusters_distances_other_work= df_clusters_distances_other_work.groupby('uid')['cluster'].apply(list).reset_index(name='other_cluster_work_dist')

## select ppl having more than one other in their schedule

In [290]:
modified_ppl = df_act_schedule_0[df_act_schedule_0.purpose=='Other'].groupby(['uid']).size().reset_index(name='count')
modified_ppl = modified_ppl['uid'][modified_ppl['count']>1].unique()

In [291]:

ppl_other =df_act_schedule_0[df_act_schedule_0.uid.isin(modified_ppl)]

ppl_other= ppl_other.sort_values(by=['uid','act_seq'])

In [292]:
# ppl_other = pd.merge(ppl_other, df_clusters_distances[['uid','cluster','distance','work_dist']], on= ['uid','cluster'], how='left')

In [293]:
ppl_other = ppl_other[['uid', 'act_seq_simp_'+day, 'cluster']]

In [294]:
ppl_other_cluster= ppl_other.groupby('uid')['cluster'].apply(list).reset_index(name='cluster_list')

ppl_other = ppl_other.drop_duplicates(subset=['uid'])


ppl_other = pd.merge(ppl_other[['uid', 'act_seq_simp_'+day]],ppl_other_cluster, on='uid' )
ppl_other = pd.merge(ppl_other,df_clusters_distances_other_home, on='uid' ,how='left')
ppl_other = pd.merge(ppl_other,df_clusters_distances_other_work, on='uid' ,how='left')



In [295]:
def seq_modifier(data):
    alist = list(data['act_seq_simp_'+day])
    cluster_list_new = data['cluster_list'].copy()
    home_other = data['other_cluster_dist'].copy()
    work_other = data['other_cluster_work_dist'].copy()
    anc_type = None  # Initialize anc_type before the loop
    for i in range(0, len(alist) - 1, 1):
        if alist[i] in ['Home','Work']:
            # anc_type='Home'
            anc_type=alist[i]
        # elif alist[i] in ['Work']:
        #     anc_type='Work'
        elif alist[i] in ['Other']:
            if anc_type=='Home':
                cluster_list_new.insert(i, home_other[0])
                del cluster_list_new[i+1]
                del home_other[0]
                del work_other[0]
            elif anc_type=='Work':
                cluster_list_new.insert(i, work_other[0])
                del cluster_list_new[i+1]
                del home_other[0]
                del work_other[0]


    data['cluster_list_new'] = cluster_list_new
    return data

tqdm.pandas()
ppl_other = ppl_other.apply(lambda row: seq_modifier(row), axis=1)

In [296]:
ppl_other = ppl_other[['uid','cluster_list_new']]

ppl_other.sort_values(by=['uid'], inplace=True)

ppl_other = ppl_other.explode('cluster_list_new')

In [297]:
df_act_schedule_0_modified =df_act_schedule_0[df_act_schedule_0.uid.isin(modified_ppl)]

df_act_schedule_0_modified = df_act_schedule_0_modified.sort_values(by=['uid','act_seq'])

In [298]:
#df_act_schedule_0_modified[df_act_schedule_0_modified.uid=='fffd8cc1-2544-4393-9011-edd0a1f75e70-1']

In [299]:
df_act_schedule_0_modified['cluster']= ppl_other['cluster_list_new'].values

In [300]:
#df_act_schedule_0_modified[df_act_schedule_0_modified.uid=='fffd8cc1-2544-4393-9011-edd0a1f75e70-1']

In [301]:
df_act_schedule_0_modified.drop(columns=['X', 'Y','lat', 'lng'], inplace=True)

In [302]:
df_act_schedule_0_modified = pd.merge(df_act_schedule_0_modified, df_clusters[['uid','cluster','X', 'Y','cluster_lat', 'cluster_lng']], on=['uid','cluster'], how='left')

In [303]:
df_act_schedule_0_modified.rename(columns={'cluster_lat':'lat', 'cluster_lng':'lng'}, inplace=True)

In [304]:
df_act_schedule_0_not_modified =df_act_schedule_0[~df_act_schedule_0.uid.isin(modified_ppl)]

In [305]:
df_act_schedule = pd.concat([df_act_schedule_0_not_modified, df_act_schedule_0_modified], ignore_index=True)

In [306]:
file = r'.\dbs\twins\%s-schedule-%s.pkl'%(filename[26:31],day)

df_act_schedule.to_pickle(file)